In [1]:
import pandas as pd
import pyspark as spark
from pyspark.sql import SparkSession
import re
from pyspark.sql.types import ArrayType, StringType, FloatType
import struct
from pyspark.sql.functions import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("Spark - Text Processing") \
    .getOrCreate()

In [3]:
df = spark.read.csv("/FileStore/tables/hacker_news_sample.csv", header=True)
rawdata = df.select("text", "by").filter(df['type'] == 'story').filter(df['text'] != "null")

In [4]:
rawdata.count()

Out[4]: 53178

In [5]:
def cleanup_text(record):
    text = record
    words = text.split()
    
    # Default list of Stopwords
    stopwords_core = ['a', u'about', u'above', u'after', u'again', u'against', u'all', u'am', u'an', u'and', u'any', u'are', u'arent', u'as', u'at', 
    u'be', u'because', u'been', u'before', u'being', u'below', u'between', u'both', u'but', u'by', 
    u'can', 'cant', 'come', u'could', 'couldnt', 
    u'd', u'did', u'didn', u'do', u'does', u'doesnt', u'doing', u'dont', u'down', u'during', 
    u'each', 
    u'few', 'finally', u'for', u'from', u'further', 
    u'had', u'hadnt', u'has', u'hasnt', u'have', u'havent', u'having', u'he', u'her', u'here', u'hers', u'herself', u'him', u'himself', u'his', u'how', 
    u'i', u'if', u'in', u'into', u'is', u'isnt', u'it', u'its', u'itself', 
    u'just', 
    u'll', 
    u'm', u'me', u'might', u'more', u'most', u'must', u'my', u'myself', 
    u'no', u'nor', u'not', u'now', 
    u'o', u'of', u'off', u'on', u'once', u'only', u'or', u'other', u'our', u'ours', u'ourselves', u'out', u'over', u'own', 
    u'r', u're', 
    u's', 'said', u'same', u'she', u'should', u'shouldnt', u'so', u'some', u'such', 
    u't', u'than', u'that', 'thats', u'the', u'their', u'theirs', u'them', u'themselves', u'then', u'there', u'these', u'they', u'this', u'those', u'through', u'to', u'too', 
    u'under', u'until', u'up', 
    u'very', 
    u'was', u'wasnt', u'we', u'were', u'werent', u'what', u'when', u'where', u'which', u'while', u'who', u'whom', u'why', u'will', u'with', u'wont', u'would', 
    u'y', u'you', u'your', u'yours', u'yourself', u'yourselves']
    
    # Custom List of Stopwords - Add your own here
    stopwords_custom = ['']
    stopwords = stopwords_core + stopwords_custom
    stopwords = [word.lower() for word in stopwords]    

    text_out = [re.sub(r'<[^>]*>',' ',word) for word in words] # Remove html tags
    text_out = [re.sub(r'http://\S+|https://\S','',word) for word in words] #remove valid links
    text_out = [re.sub(r'http:\S+|https://\S+',' ',word) for word in words] #remove invalid links
    
    text_out = [re.sub('[^a-zA-Z0-9]','',word) for word in words]                                       # Remove special characters
    text_out = [word.lower() for word in text_out if len(word)>2 and word.lower() not in stopwords]     # Remove stopwords and words under X length
    return text_out

udf_cleantext = udf(cleanup_text , ArrayType(StringType()))
clean_text = rawdata.withColumn("words", udf_cleantext(rawdata['text']))

In [6]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer

#hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
#featurizedData = hashingTF.transform(clean_text)

# Term Frequency Vectorization  - Option 2 (CountVectorizer)    : 
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures", vocabSize = 1000)
cvmodel = cv.fit(clean_text)
featurizedData = cvmodel.transform(clean_text)

vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData) # TFIDF

In [7]:
# rescaledData.select('features').show(10)

In [8]:
from pyspark.ml.clustering import LDA

lda = LDA(k=25, seed=123, optimizer="em", featuresCol="features")

ldamodel = lda.fit(rescaledData)

ldatopics = ldamodel.describeTopics()
#ldatopics.show(25)

def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(50,False)

+-----+---------------------------------------------------------------------------------------------------+
topic|topic_desc |
+-----+---------------------------------------------------------------------------------------------------+
0 |[like, get, people, one, new, time, work, know, 2012, domain] |
1 |[like, get, people, work, time, know, one, ix27m, want, medical] |
2 |[like, get, time, one, people, work, know, free, good, ix27m] |
3 |[text, like, games, use, get, one, new, people, know, online] |
4 |[dvd, hair, music, car, player, india, mac, mobile, video, videos] |
5 |[iphone, application, like, development, web, app, get, time, people, use] |
6 |[ipad, like, medical, get, web, one, time, people, new, use] |
7 |[like, web, one, get, use, people, time, know, hosting, work] |
8 |[ipad, like, get, one, people, app, use, time, know, new] |
9 |[seo, like, get, web, one, people, new, work, know, time] |
10 |[essay, writing, students, write, like, ix27m, work, good, get, people] |
11 |[samsung, memory, like, camera, time, new, get, one, people, work] |
12 |[shoes, air, nike, free, sale, cheap, running, price, run, new] |
13 |[movie, watch, like, people, one, get, time, film, movies, online] |
14 |[insurance, online, designer, wedding, shopping, indian, designs, password, affordable, collection]|
15 |[seo, like, get, people, time, work, know, one, want, ix27m] |
16 |[water, password, like, get, one, people, time, use, know, new] |
17 |[property, battery, investors, laptop, like, one, get, time, work, people] |
18 |[desktop, yang, dan, wallpapers, plus, photos, hot, beautiful, download, images] |
19 |[london, like, get, company, work, people, business, ix27m, time, startup] |
20 |[like, get, people, one, time, know, web, new, work, want] |
21 |[estate, real, paper, like, get, one, new, web, know, people] |
22 |[like, work, people, get, ix27m, company, time, one, know, business] |
23 |[like, get, people, app, one, time, web, use, new, know] |
24 |[pdf, like, work, files, net, get, time, people, ix27m, one] |
+-----+---------------------------------------------------------------------------------------------------+

In [9]:
ldaResults = ldamodel.transform(rescaledData)
ldaResults.show()

+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+
 text| by| words| rawFeatures| features| topicDistribution|
+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+
&gt;<i>which lead...| coldtea|[gtiwhich, leads,...|(1000,[4,5,11,13,...|(1000,[4,5,11,13,...|[0.03177307781880...|
I would like to p...| etanol|[like, point, cou...|(1000,[0,1,15,17,...|(1000,[0,1,15,17,...|[0.04794108984711...|
<i>Our msbuild im...| Locke1689|[iour, msbuild, i...|(1000,[11,185,209...|(1000,[11,185,209...|[0.04059060017236...|
No matter how awf...| miloshadzic|[matter, awful, i...|(1000,[2,14,17,24...|(1000,[2,14,17,24...|[0.03814808995895...|
The existence of ...| salsakran|[existence, way, ...|(1000,[6,9,12,16,...|(1000,[6,9,12,16,...|[0.03921469113649...|
The actual Intern...| paulsutter|[actual, internet...|(1000,[4,12,17,20...|(1000,[4,12,17,20...|[0.04318048618519...|
I want to know ho...| sitkack|[want, know, ants...|(1000,[17,20,90,9...|(1000,[17,20,90,9...|[0.04180151140018...|
French is supreme...| cjsthompson|[french, supremel...|(1000,[4,9,55,77,...|(1000,[4,9,55,77,...|[0.03844060651348...|
I actually went f...| evgen|[actually, went, ...|(1000,[22,29,33,4...|(1000,[22,29,33,4...|[0.03919991187477...|
First impression ...| adventured|[first, impressio...|(1000,[6,40,68,44...|(1000,[6,40,68,44...|[0.03874310177536...|
It&#x27;s not exa...| jhanschoo|[itx27s, exactly,...|(1000,[4,15,50,66...|(1000,[4,15,50,66...|[0.03748589276912...|
Thanks. I choose ...| greggman|[thanks, choose, ...|(1000,[3,5,12,19,...|(1000,[3,5,12,19,...|[0.03542132446360...|
I used Sybase ASE...|lobster_johnson|[used, sybase, as...|(1000,[7,14,31,41...|(1000,[7,14,31,41...|[0.03559449459484...|
&gt;They&#x27;ve ...| shrimp_emoji|[gttheyx27ve, tri...|(1000,[2,14,52,25...|(1000,[2,14,52,25...|[0.03671933493508...|
If there's a high...| cturner|[theres, high, ch...|(1000,[6,32,77,17...|(1000,[6,32,77,17...|[0.04063224237200...|
"Actually the def...| idlewords|[actually, defini...|(1000,[29,231,329...|(1000,[29,231,329...|[0.03972217960365...|
&#62; If the appl...| eru|[application, bec...|(1000,[0,2,9,12,1...|(1000,[0,2,9,12,1...|[0.03551371668978...|
Looks like the 10...| pawadu|[looks, like, 10k...|(1000,[0,83,235,8...|(1000,[0,83,235,8...|[0.04006670737613...|
Definitely has th...| napierzaza|[definitely, slan...|(1000,[0,1,24,52,...|(1000,[0,1,24,52,...|[0.04000943793024...|
Funny I had just ...| tdicola|[funny, switched,...|(1000,[33,46,65,1...|(1000,[33,46,65,1...|[0.03905103120181...|
+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [10]:
def breakout_array(index_number, record):
    vectorlist = record.tolist()
    return vectorlist[index_number]

udf_breakout_array = udf(breakout_array, FloatType())

# Extract document weights for Topics 12 and 20
enrichedData = ldaResults.withColumn("Topic_12", udf_breakout_array(lit(12), ldaResults.topicDistribution)).withColumn("topic_20", udf_breakout_array(lit(20), ldaResults.topicDistribution)) 

In [11]:
enrichedData.select("text", "by", "Topic_12").sort(desc("Topic_12")).show(5)

+--------------------+---------+----------+
 text| by| Topic_12|
+--------------------+---------+----------+
And with this one...| pilif|0.42901105|
"As I see it HATE...|friendzis|0.40850642|
"&#62;<i>""By Dec...| brudgers|0.38949075|
"<i>All the Linux...| dkarl|0.38648403|
In the US I worke...| furyg3|0.36013252|
+--------------------+---------+----------+
only showing top 5 rows

In [12]:
# enrichedData.select("text", "Topic_12").filter(enrichedData['by'] == 'karl_gluck').sort(desc("Topic_12")).show(5)

+--------------------+----------+
 text| Topic_12|
+--------------------+----------+
"Hi everyone! I'...|0.50739044|
+--------------------+----------+

In [13]:
data = enrichedData.toPandas()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1193872435447101> in <module> 
 ----> 1 data = enrichedData . toPandas ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in toPandas (self) 
 2197 
 2198 # Below is toPandas without Arrow optimization. 
 -> 2199 pdf = pd . DataFrame . from_records ( self . collect ( ) , columns = self . columns ) 
 2200 
 2201 dtype = { } 

 /databricks/spark/python/pyspark/sql/dataframe.py in collect (self) 
 547 # Default path used in OSS Spark / for non-DF-ACL clusters: 
 548 with SCCallSiteSync ( self . _sc ) as css : 
 --> 549 sock_info = self . _jdf . collectToPython ( ) 
 550 return list ( _load_from_socket ( sock_info , BatchedSerializer ( PickleSerializer ( ) ) ) ) 
 551 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o8747.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 910.0 failed 1 times, most recent failure: Lost task 0.0 in stage 910.0 (TID 1349, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2349)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2349)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2581)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2529)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2517)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:897)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2280)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2378)
	at org.apache.spark.sql.execution.collect.Collector.runSparkJobs(Collector.scala:245)
	at org.apache.spark.sql.execution.collect.Collector.collect(Collector.scala:280)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:80)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:86)
	at org.apache.spark.sql.execution.ResultCacheManager.getOrComputeResult(ResultCacheManager.scala:508)
	at org.apache.spark.sql.execution.ResultCacheManager.getOrComputeResult(ResultCacheManager.scala:4

In [14]:
max_idx = []
for i in range(len(data2['topicDistribution'])):
  tempHigh = -100000
  tempHighIdx = -1
  for j in range(len(data2['topicDistribution'][i])):
    if data2['topicDistribution'][i][j] > tempHigh:
      tempHigh = data2['topicDistribution'][i][j]
      tempHighIdx = j
  
  max_idx.append(tempHighIdx)

data2['maxTopic'] = max_idx

In [15]:
data2['maxTopic'].value_counts()
mostPopular = [1,4,0,15]

In [16]:
# Visualization

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
import numpy as np

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

fig, axes = plt.subplots(2,2,figsize=(16,14), dpi=160, sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):    
    df_dominant_topic_sub = data2.loc[data2.maxTopic == mostPopular[i], :]
    doc_lens = [len(d) for d in df_dominant_topic_sub.words]
    
    ax.hist(doc_lens, bins = 1000, color=cols[i])
    ax.tick_params(axis='y', labelcolor=cols[i], color=cols[i])
    sns.kdeplot(doc_lens, color="black", shade=False, ax=ax.twinx())
    ax.set(xlim=(0, 1000), xlabel='Document Word Count')
    ax.set_ylabel('Number of Documents', color=cols[i])
    ax.set_title('Topic: '+str(mostPopular[i]), fontdict=dict(size=16, color=cols[i]))

fig.tight_layout()
fig.subplots_adjust(top=0.90)
plt.xticks(np.linspace(0,1000,9))
fig.suptitle('Distribution of Document Word Counts by Dominant Topic', fontsize=22)
display()